<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Xinhe_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 14.2 MB/s 


In [5]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [ ]:
expression.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
01_a_D_f_2.CEL,9.521934,7.453767,7.045636,7.743690,7.728274,8.081243,6.927978,9.294152,8.888816,8.290944,...,7.065597,6.803698,6.631766,6.981474,7.003319,6.999630,7.005495,6.864895,7.030665,14.394582
01_a_I_f_2.CEL,10.930684,9.455482,9.233291,9.777128,9.612399,9.845444,9.149639,10.580062,11.505571,9.846817,...,9.366604,9.067484,8.945584,9.118070,9.137492,9.067202,9.129742,9.256544,9.171725,15.494106
01_a_M_f_1.CEL,6.852731,5.298974,5.033266,6.040661,5.890083,5.810144,5.098923,6.240855,7.280948,6.124910,...,5.374457,5.165619,5.032617,5.382904,5.509690,5.113946,5.273243,5.321357,5.321636,12.856782
01_c_D_f_1.CEL,7.285181,6.258114,6.119443,6.631768,6.744592,6.589478,6.156638,7.417750,8.663882,8.203275,...,6.332162,6.278864,6.108778,6.341791,6.339638,6.167335,6.170734,6.734936,6.275118,13.683484
01_c_I_f_2.CEL,11.224543,9.800931,9.407753,10.113212,9.871853,10.495533,9.449701,11.084619,11.969619,10.479006,...,9.443910,9.242560,9.272848,9.435175,9.448927,9.395768,9.306998,9.863770,9.404475,15.494106


In [6]:
genes.head()

,sym,chrom
rownames,,
1000_at,MAPK3,16
1001_at,TIE1,1
1002_f_at,CYP2C19,10
1003_s_at,CXCR5,11
1004_at,CXCR5,11


In [8]:
samples.head()

,patient,sex,region,lab,chip.version
rownames,,,,,
01_a_D_f_2.CEL,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,patient_01,female,cerebellum,Irvine,v2


In [13]:
# borrowed some code from Vinod
express_samp = pd.concat([expression, samples], axis=1)

express_samp.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at,patient,sex,region,lab,chip.version
01_a_D_f_2.CEL,9.521934,7.453767,7.045636,7.743690,7.728274,8.081243,6.927978,9.294152,8.888816,8.290944,...,6.999630,7.005495,6.864895,7.030665,14.394582,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,10.930684,9.455482,9.233291,9.777128,9.612399,9.845444,9.149639,10.580062,11.505571,9.846817,...,9.067202,9.129742,9.256544,9.171725,15.494106,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,6.852731,5.298974,5.033266,6.040661,5.890083,5.810144,5.098923,6.240855,7.280948,6.124910,...,5.113946,5.273243,5.321357,5.321636,12.856782,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,7.285181,6.258114,6.119443,6.631768,6.744592,6.589478,6.156638,7.417750,8.663882,8.203275,...,6.167335,6.170734,6.734936,6.275118,13.683484,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,11.224543,9.800931,9.407753,10.113212,9.871853,10.495533,9.449701,11.084619,11.969619,10.479006,...,9.395768,9.306998,9.863770,9.404475,15.494106,patient_01,female,cerebellum,Irvine,v2


In [44]:
express_samp_UM = express_samp[express_samp.lab == "Michigan"]

p_values = pd.DataFrame()

# for each tissue, permute the 10 outcomes, 
# use the difference-in-means as the test statistic
for tissue in express_samp_UM["region"].unique():
  print(tissue)
  express_samp_UM_new = express_samp_UM[(express_samp_UM.region == tissue)]
  express_samp_UM_new = pd.concat([express_samp_UM_new, pd.DataFrame(columns=['perm'])], axis=1)

  df_male = express_samp_UM_new[express_samp_UM_new.sex == "male"]
  df_female = express_samp_UM_new[express_samp_UM_new.sex == "female"]

  male_mean = df_male.mean()
  female_mean = df_female.mean()

  diff = pd.DataFrame(male_mean - female_mean)
  for comb in combinations(range(10), 5):
    #print(comb)
    express_samp_UM_new.perm = [0] * 10

    express_samp_UM_new.perm.iloc[list(comb)] = 1

    df_male = express_samp_UM_new[express_samp_UM_new.perm == 1]
    df_female = express_samp_UM_new[express_samp_UM_new.perm == 0]

    male_mean = df_male.mean()
    female_mean = df_female.mean()

    diff = pd.concat([diff, male_mean - female_mean], axis=1)
    #print(diff.shape)
  
  # compare columns and get p-values
  diff = diff.abs()
  for j in range(252):
    diff.iloc[:,j+1] = diff.iloc[:,0] <= diff.iloc[:,j+1]
  p_values = pd.concat([p_values, diff.iloc[:,1:].mean(axis=1)], axis=1)

p_values.head()

A.C. cortex


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: Dropping of nu

cerebellum
D.L.P.F. cortex


,0,0,0
1000_at,0.476190,0.563492,0.785714
1001_at,0.619048,0.769841,0.579365
1002_f_at,0.341270,0.476190,0.888889
1003_s_at,0.365079,0.706349,0.753968
1004_at,0.246032,0.555556,0.825397


In [48]:
sum(p_values.mean(axis=1) <= 0.05) # 3
p_values[p_values.mean(axis=1) <= 0.2]

,0,0,0
266_s_at,0.087302,0.230159,0.142857
35885_at,0.103175,0.380952,0.111111
36321_at,0.031746,0.309524,0.126984
37583_at,0.055556,0.206349,0.261905
38182_at,0.039683,0.380952,0.158730
38355_at,0.007937,0.015873,0.007937
41214_at,0.007937,0.015873,0.007937
969_s_at,0.103175,0.222222,0.190476
perm,0.000000,0.000000,0.000000
